In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import permutations
from datetime import date

In [2]:
data = pd.read_csv('movie_bd_v5.xls')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
613,tt0411477,85000000,160388063,Hellboy II: The Golden Army,Ron Perlman|Selma Blair|Jeffrey Tambor|Doug Jo...,Guillermo del Toro,Saving the world is a hell of a job.,In this continuation to the adventure of the d...,120,Adventure|Fantasy|Science Fiction,Universal Pictures|Dark Horse Entertainment|La...,7/11/2008,6.4,2008
1392,tt0358273,28000000,186438883,Walk the Line,Joaquin Phoenix|Reese Witherspoon|Ginnifer Goo...,James Mangold,Love is a burning thing.,A chronicle of country music legend Johnny Cas...,136,Drama|Music|Romance,Tree Line Films|Konrad Pictures|Catfish Produc...,9/13/2005,7.0,2005
731,tt1570728,50000000,142851197,"Crazy, Stupid, Love.",Steve Carell|Julianne Moore|Ryan Gosling|Emma ...,Glenn Ficarra|John Requa,This Is Stupid,Cal Weaver is living the American dream. He ha...,118,Comedy|Drama|Romance,Warner Bros.,7/29/2011,6.9,2011
1505,tt0466909,25000000,119188334,The Omen,Liev Schreiber|Julia Stiles|Seamus Davey-Fitzp...,John Moore,A new generation of terror,A diplomatic couple adopts the son of the devi...,110,Drama|Horror|Thriller,Twentieth Century Fox Film Corporation|11:11 M...,6/6/2006,5.2,2006
1659,tt0331953,8500000,5001655,Club Dread,Kevin Heffernan|Jay Chandrasekhar|Paul Soter|E...,Jay Chandrasekhar,A vacation to die for,When a serial killer interrupts the fun at the...,104,Comedy|Horror,Fox Searchlight Pictures|Broken Lizard Industr...,2/27/2004,5.1,2004


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [39]:
def company_name_correction(name):
    """Преобразование имен компаний к единому имени"""    
    if name.find('Warner Bros') >= 0:   # все компании, вроде 'Warner Bros.' 'Warner Bros. Pictures' и т.д.
        return 'Warner Bros'            # будут иметь общее имя 'Warner Bros'
    else:
        return name                     # имена других компаний оставляем как есть
    
    
answers = {} # создадим словарь для ответов
    
# ниже производится добавление вспомогательных столбцов в датафрейм data
# вычисление прибыли фильма
data['profit'] = data['revenue'] - data['budget']
        
# преобразование строки жанров в список жанров
data['genres_list'] = data['genres'].str.split('|')
display(data['genres_list'])

# то же самое для строки актеров
data['cast_list'] = data['cast'].str.split('|')
display(data['cast_list'])

# то же самое для компаний, при этом производится исправление имени компании Warner Bros
data['companies_list'] = data['production_companies'].str.split('|').apply(lambda lst: [company_name_correction(name) for name in lst])
display(data['companies_list'])

# преобразование в формат python date (с перестановкой местами дня, месяца, года)
data['release_pydate'] = data['release_date'].str.split('/').apply(lambda d: date(int(d[2]), int(d[0]), int(d[1])))
display(data['release_pydate'])

0       [Action, Adventure, Science Fiction, Thriller]
1       [Action, Adventure, Science Fiction, Thriller]
2               [Adventure, Science Fiction, Thriller]
3        [Action, Adventure, Science Fiction, Fantasy]
4                            [Action, Crime, Thriller]
                             ...                      
1884              [Adventure, Action, Science Fiction]
1885                         [Horror, Family, Foreign]
1886    [Adventure, Animation, Action, Comedy, Family]
1887                                   [Comedy, Drama]
1888                                        [Thriller]
Name: genres_list, Length: 1889, dtype: object

0       [Chris Pratt, Bryce Dallas Howard, Irrfan Khan...
1       [Tom Hardy, Charlize Theron, Hugh Keays-Byrne,...
2       [Shailene Woodley, Theo James, Kate Winslet, A...
3       [Harrison Ford, Mark Hamill, Carrie Fisher, Ad...
4       [Vin Diesel, Paul Walker, Jason Statham, Miche...
                              ...                        
1884    [Patrick Stewart, Hugh Jackman, Ian McKellen, ...
1885    [Richard E. Grant, Jonathan Lipnicki, Jim Cart...
1886    [Rene Russo, Jason Alexander, Piper Perabo, Ra...
1887    [Meg Ryan, Diane Keaton, Lisa Kudrow, Walter M...
1888    [Susan Ward, Lori Heuring, Matthew Settle, Nat...
Name: cast_list, Length: 1889, dtype: object

0       [Universal Studios, Amblin Entertainment, Lege...
1       [Village Roadshow Pictures, Kennedy Miller Pro...
2       [Summit Entertainment, Mandeville Films, Red W...
3           [Lucasfilm, Truenorth Productions, Bad Robot]
4       [Universal Pictures, Original Film, Media Righ...
                              ...                        
1884    [Twentieth Century Fox Film Corporation, Donne...
1885                                    [New Line Cinema]
1886    [Universal Pictures, Capella International, KC...
1887    [Laurence Mark Productions, Columbia Pictures ...
1888                                        [Warner Bros]
Name: companies_list, Length: 1889, dtype: object

0       2015-06-09
1       2015-05-13
2       2015-03-18
3       2015-12-15
4       2015-04-01
           ...    
1884    2000-07-13
1885    2000-10-27
1886    2000-06-30
1887    2000-02-16
1888    2000-07-19
Name: release_pydate, Length: 1889, dtype: object

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [5]:
answers['1'] = data.loc[data['budget'].idxmax()]['original_title']
display(answers['1'])

'Pirates of the Caribbean: On Stranger Tides'

# 2. Какой из фильмов самый длительный (в минутах)?

In [6]:
answers['2'] = data.loc[data['runtime'].idxmax()]['original_title']
display(answers['2'])

'Gods and Generals'

# 3. Какой из фильмов самый короткий (в минутах)?





In [7]:
answers['3'] = data.loc[data['runtime'].idxmin()]['original_title']
display(answers['3'])

'Winnie the Pooh'

# 4. Какова средняя длительность фильмов?


In [8]:
answers['4'] = np.round(data['runtime'].mean())
display(answers['4'])

110.0

# 5. Каково медианное значение длительности фильмов? 

In [9]:
answers['5'] = data['runtime'].median()
display(answers['5'])

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [10]:
answers['6'] = data.loc[data['profit'].idxmax()]['original_title']
display(answers['6'])

'Avatar'

# 7. Какой фильм самый убыточный? 

In [11]:
answers['7'] = data.loc[data['profit'].idxmin()]['original_title']
display(answers['7'])

'The Lone Ranger'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [12]:
answers['8'] = len(data[data['profit'] > 0])
display(answers['8'])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [13]:
answers['9'] = data.loc[data[data['release_pydate'].apply(lambda d: d.year == 2008)]['revenue'].idxmax()]['original_title']
display(answers['9'])

'The Dark Knight'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [14]:
answers['10'] = data.loc[data[data['release_pydate'].apply(lambda d: d.year in [2012, 2013, 2014])]['profit'].idxmin()]['original_title']
display(answers['10'])

'The Lone Ranger'

# 11. Какого жанра фильмов больше всего?

In [16]:
genre_counter = Counter() # счетчик фильмов для каждого жанра по отдельности
for genres_list in data['genres_list']:
    for genre in genres_list:
        genre_counter[genre] += 1

answers['11'] = genre_counter.most_common(1)[0][0]
display(answers['11'])

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [17]:
profit_genre_counter = Counter() # счетчик прибыльных фильмов для каждого жанра по отдельности
for genres_list in data[data['profit'] > 0]['genres_list']:
    for genre in genres_list:
        profit_genre_counter[genre] += 1

answers['12'] = profit_genre_counter.most_common(1)[0][0]
display(answers['12'])

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [18]:
answers['13'] = data.groupby(['director'])['revenue'].sum().idxmax()
display(answers['13'])

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [19]:
answers['14'] = data[data['genres_list'].apply(lambda g_list: 'Action' in g_list)].groupby(['director']).apply(len).idxmax()
display(answers['14'])

'Michael Bay'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [20]:
actor_revenue_counter = Counter() # счетчик суммарных кассовых расходов для каждого актера, снимавшегося в 2012 году
for key, cast_list in data[data['release_pydate'].apply(lambda d: d.year == 2012)]['cast_list'].items():
    for actor in cast_list:
        actor_revenue_counter[actor] += data['revenue'][key]

answers['15'] = actor_revenue_counter.most_common(1)[0][0]
display(answers['15'])

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [21]:
actor_film_counter = Counter() # счетчик высокобюджетных фильмов для каждого актера
for cast_list in data[data['budget'] > data['budget'].mean()]['cast_list']:
    for actor in cast_list:
        actor_film_counter[actor] += 1

answers['16'] = actor_film_counter.most_common(1)[0][0]
display(answers['16'])

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [22]:
nc_genres_counter = Counter() # счетчик жанров Николаса Кейджа
for genres_list in data[data['cast_list'].apply(lambda c_list: 'Nicolas Cage' in c_list)]['genres_list']:
    for genre in genres_list:
        nc_genres_counter[genre] += 1
    
answers['17'] = nc_genres_counter.most_common(1)[0][0]
display(answers['17'])

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [23]:
answers['18'] = data.loc[data[data['companies_list'].apply(lambda c_list: 'Paramount Pictures' in c_list)]['profit'].idxmin()]['original_title']
display(answers['18'])

'K-19: The Widowmaker'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [24]:
answers['19'] = data.groupby(by=data['release_pydate'].apply(lambda d: d.year))['revenue'].sum().idxmax()
display(answers['19'])

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [25]:
answers['20'] = data[data['companies_list'].apply(lambda c_list: 'Warner Bros' in c_list)].groupby(by=data['release_pydate'].apply(lambda d: d.year))['profit'].sum().idxmax()
display(answers['20'])

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [26]:
answers['21'] = data.groupby(by=data['release_pydate'].apply(lambda d: d.month)).apply(len).idxmax()
display(answers['21'])

9

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [27]:
answers['22'] = len(data[data['release_pydate'].apply(lambda d: d.month in [6, 7, 8])])
display(answers['22'])

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [28]:
answers['23'] = data[data['release_pydate'].apply(lambda d: d.month in [12, 1, 2])].groupby(['director']).apply(len).idxmax()
display(answers['23'])

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [29]:
film_counter = Counter() # счетчик количества фильмов для каждой компании
title_length_counter = Counter() # счетчик суммарной длины названий фильмов для каждой компании
for key, companies_list in data['companies_list'].items():
    for company in companies_list:
        film_counter[company] += 1
        title_length_counter[company] += len(data.loc[key]['original_title'])
        
# заполняется список списков средних длин названий фильмов для каждой компании
mean_title_length = [] 
for company in film_counter.keys(): # в счетчиках уже все компании посчитаны
    mean_title_length.append([company, title_length_counter[company] / film_counter[company]])

#список списков => датафрейм    
mtl_df = pd.DataFrame(mean_title_length, columns=['company', 'mean_title_length']) 

answers['24'] = mtl_df[mtl_df['mean_title_length'] == mtl_df['mean_title_length'].max()]['company'].values[0]
display(answers['24'])

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [30]:
overview_length_counter = Counter() # счетчик количества слов в описании фильма для каждой компании
# далее счетчик film_counter не вычисляется, используется результат вычисления в предыдущей задаче
for key, companies_list in data['companies_list'].items():
    for company in companies_list:
        overview_length_counter[company] += len(data.loc[key]['overview'].split())
        
# заполняется список списков средних количеств слов в описании для каждой компании
mean_overview_length = [] 
for company in film_counter.keys(): # в счетчиках уже все компании посчитаны
    mean_overview_length.append([company, overview_length_counter[company] / film_counter[company]])

#список списков => датафрейм    
mol_df = pd.DataFrame(mean_overview_length, columns=['company', 'mean_overview_length']) 

answers['25'] = mol_df[mol_df['mean_overview_length'] == mol_df['mean_overview_length'].max()]['company'].values[0]
display(answers['25'])

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [38]:
one_percent_count = int(np.floor(0.01 * len(data))) # количество лучших фильмов, входящих в 1 процент
print(one_percent_count)

answers['26'] = data.sort_values(['vote_average'], ascending=False)['original_title'][:one_percent_count].values
display(answers['26'])

18


array(['The Dark Knight', 'Interstellar', 'The Imitation Game',
       'Inside Out', 'Room', 'The Wolf of Wall Street', 'Gone Girl',
       '12 Years a Slave', 'Guardians of the Galaxy',
       'The Lord of the Rings: The Return of the King', 'Memento',
       'Inception', 'The Pianist', 'The Grand Budapest Hotel', 'Her',
       'Spotlight', 'Big Hero 6', 'The Fault in Our Stars'], dtype=object)

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [32]:
pairs_counter = Counter() # счетчик всех пар актеров
for cast_list in data['cast_list']:
    for pairs in permutations(cast_list, 2):
        pairs_counter[pairs] += 1

answers['27'] = pairs_counter.most_common(1)[0][0]
display(answers['27'])

('Daniel Radcliffe', 'Rupert Grint')

# Submission

In [33]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': 110.0,
 '5': 107.0,
 '6': 'Avatar',
 '7': 'The Lone Ranger',
 '8': 1478,
 '9': 'The Dark Knight',
 '10': 'The Lone Ranger',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Michael Bay',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': 2015,
 '20': 2014,
 '21': 9,
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': array(['The Dark Knight', 'Interstellar', 'The Imitation Game',
        'Inside Out', 'Room', 'The Wolf of Wall Street', 'Gone Girl',
        '12 Years a Slave', 'Guardians of the Galaxy',
        'The Lord of the Rings: The Return of the King', 'Memento',
        'Inception', 'The Pianist', 'The Grand Budapest Hotel', 'Her',
        'Spotlight', 'Big Hero 6', 'The Fault in Our Stars'], dtype=object),
 '27': ('Daniel Radcliffe', 'Rupert Grint'

In [34]:
# и убедиться что ни чего не пропустил)
len(answers)

27